# Generating RP images

In [5]:
import os
import numpy as np
import pandas as pd
from pyts.image import RecurrencePlot
from sklearn.preprocessing import MinMaxScaler

# Define paths
dataset_paths = {
    "trial1": "D:\\UP-FALL",
    "trial2": "D:\\UP-FALL\\trial2",
    "trial3": "D:\\UP-FALL\\trial3"
}
output_base_path = "D:\\UP-FALL\\v2"
os.makedirs(output_base_path, exist_ok=True)

# Define column names
columns = [
    "Timestamp",
    "AnkleAccelerometer_x", "AnkleAccelerometer_y", "AnkleAccelerometer_z",
    "AnkleAngularVelocity_x", "AnkleAngularVelocity_y", "AnkleAngularVelocity_z",
    "AnkleLuminosity",
    # Other sensors are ignored for this task
    "RightPocketAccelerometer_x", "RightPocketAccelerometer_y", "RightPocketAccelerometer_z",
    "RightPocketAngularVelocity_x", "RightPocketAngularVelocity_y", "RightPocketAngularVelocity_z",
    "RightPocketLuminosity",
    "BeltAccelerometer_x", "BeltAccelerometer_y", "BeltAccelerometer_z",
    "BeltAngularVelocity_x", "BeltAngularVelocity_y", "BeltAngularVelocity_z",
    "BeltLuminosity",
    "NeckAccelerometer_x", "NeckAccelerometer_y", "NeckAccelerometer_z",
    "NeckAngularVelocity_x", "NeckAngularVelocity_y", "NeckAngularVelocity_z",
    "NeckLuminosity",
    "WristAccelerometer_x", "WristAccelerometer_y", "WristAccelerometer_z",
    "WristAngularVelocity_x", "WristAngularVelocity_y", "WristAngularVelocity_z",
    "WristLuminosity",
    "BrainSensor",
    "Infrared1", "Infrared2", "Infrared3", "Infrared4", "Infrared5", "Infrared6",
    "Subject", "Activity", "Trial"
]

# Define function to generate recurrence plot images
# Define function to generate recurrence plot images
def generate_recurrence_plot(data, output_path):
    try:
        # Create RecurrencePlot object without the image_size parameter
        rp = RecurrencePlot(dimension=1, threshold='point', percentage=10)
        rp_image = rp.fit_transform(data.reshape(1, -1))[0]
        np.save(output_path, rp_image)  # Save as .npy
    except Exception as e:
        print(f"Error generating recurrence plot for {output_path}: {e}")


# Function to process sensor data
def process_sensor_data(trial, subject, activity, trial_path, output_path, window_size, overlap):
    csv_file = os.path.join(trial_path, f"Subject{subject}Activity{activity}Trial{trial}.csv")  # Corrected filename format
    
    # Check if the file exists
    if not os.path.exists(csv_file):
        print(f"File not found: {csv_file}")
        return

    # Load sensor data
    try:
        data = pd.read_csv(csv_file, skiprows=2, names=columns, usecols=range(46))
    except Exception as e:
        print(f"Error processing {csv_file}: {e}")
        return

    # Compute magnitudes for accelerometer and angular velocity
    acc_magnitude = np.sqrt(data["AnkleAccelerometer_x"]**2 + 
                            data["AnkleAccelerometer_y"]**2 + 
                            data["AnkleAccelerometer_z"]**2)
    gyro_magnitude = np.sqrt(data["AnkleAngularVelocity_x"]**2 + 
                             data["AnkleAngularVelocity_y"]**2 + 
                             data["AnkleAngularVelocity_z"]**2)

    # **Handle Empty Data**
    if acc_magnitude.empty or gyro_magnitude.empty:
        print(f"No valid sensor data for Subject {subject}, Activity {activity}, Trial {trial}")
        return

    # Normalize magnitudes
    scaler = MinMaxScaler(feature_range=(0, 1))
    acc_magnitude = scaler.fit_transform(acc_magnitude.values.reshape(-1, 1)).flatten()
    gyro_magnitude = scaler.fit_transform(gyro_magnitude.values.reshape(-1, 1)).flatten()

    # Determine window length in rows
    sampling_rate = 20  # Assuming 20 Hz
    window_rows = int(window_size * sampling_rate)
    overlap_rows = int(overlap * window_rows)

    # Create output directory
    activity_output_path = os.path.join(output_path, f"Subject{subject}_Activity{activity}_Trial{trial}")
    os.makedirs(activity_output_path, exist_ok=True)

    # Generate recurrence plots for each window
    start = 0
    window_idx = 0
    while start + window_rows <= len(acc_magnitude):
        acc_window = acc_magnitude[start:start + window_rows]
        gyro_window = gyro_magnitude[start:start + window_rows]

        if len(acc_window) == 0 or len(gyro_window) == 0:
            print(f"Skipping empty window for Subject {subject}, Activity {activity}, Trial {trial}")
            start += overlap_rows
            window_idx += 1
            continue

        # Generate and save recurrence plots
        acc_output_file = os.path.join(activity_output_path, f"window{window_idx}_acc.npy")
        gyro_output_file = os.path.join(activity_output_path, f"window{window_idx}_gyro.npy")
        try:
            generate_recurrence_plot(acc_window, acc_output_file)
            generate_recurrence_plot(gyro_window, gyro_output_file)
        except Exception as e:
            print(f"Error generating recurrence plot for {acc_output_file} or {gyro_output_file}: {e}")

        # Move the window forward
        start += overlap_rows
        window_idx += 1


# Process all trials
for trial_name, trial_path in dataset_paths.items():
    output_path = os.path.join(output_base_path, f"{trial_name}_sensor_rp_images")
    os.makedirs(output_path, exist_ok=True)

    for subject in range(1, 18):  # Subjects 1 to 17
        for activity in range(1, 12):  # Activities 1 to 11
            # Set windowing parameters based on activity duration
            if activity in [1, 2, 3, 4, 5, 9]:  # Short activities (10s)
                window_size = 3  # seconds
                overlap = 0.5
            else:  # Long activities (30-60s)
                window_size = 5  # seconds
                overlap = 0.5

            process_sensor_data(trial_name[-1], subject, activity, trial_path, output_path, window_size, overlap)

print("Recurrence plots for all trials have been created and saved.")

No valid sensor data for Subject 8, Activity 11, Trial 2
No valid sensor data for Subject 8, Activity 11, Trial 3
Recurrence plots for all trials have been created and saved.


In [6]:
import os
import numpy as np

# Define the dataset paths
dataset_paths = [
    "D:\\UP-FALL\\v2\\trial1_sensor_rp_images",
    "D:\\UP-FALL\\v2\\trial2_sensor_rp_images",
    "D:\\UP-FALL\\v2\\trial3_sensor_rp_images"
]

# Initialize variables to store min and max sizes
min_size = None
max_size = None

# Function to update min and max size
def update_min_max(size, min_size, max_size):
    if min_size is None or size < min_size:
        min_size = size
    if max_size is None or size > max_size:
        max_size = size
    return min_size, max_size

# Loop through all dataset paths
for dataset_path in dataset_paths:
    for root, _, files in os.walk(dataset_path):
        for file in files:
            if file.endswith('.npy'):
                file_path = os.path.join(root, file)
                try:
                    # Load the image
                    image = np.load(file_path)
                    # Get the size (height, width)
                    size = image.shape[:2]
                    # Update min and max sizes
                    min_size, max_size = update_min_max(size, min_size, max_size)
                except Exception as e:
                    print(f"Error processing {file_path}: {e}")

# Print the results
print("Minimum image size:", min_size)
print("Maximum image size:", max_size)

Minimum image size: (60, 60)
Maximum image size: (100, 100)


## Resizing 

In [9]:
import os
import numpy as np
from PIL import Image

# Paths
rp_image_paths = [
    "D:\\UP-FALL\\v2\\trial1_sensor_rp_images",
    "D:\\UP-FALL\\v2\\trial2_sensor_rp_images",
    "D:\\UP-FALL\\v2\\trial3_sensor_rp_images"
]

output_paths = [
    "D:\\UP-FALL\\v2\\trial1_sensor_rp_images_resized",
    "D:\\UP-FALL\\v2\\trial2_sensor_rp_images_resized",
    "D:\\UP-FALL\\v2\\trial3_sensor_rp_images_resized"
]

target_size = (32, 32)  # Target size for resizing

# Ensure output directories exist
for path in output_paths:
    os.makedirs(path, exist_ok=True)

# Function to resize images
def resize_rp_images(input_path, output_path):
    for root, _, files in os.walk(input_path):
        for file in files:
            if file.endswith('.npy'):
                input_file = os.path.join(root, file)
                # Preserve the relative path in the output directory
                relative_path = os.path.relpath(root, input_path)
                output_dir = os.path.join(output_path, relative_path)
                os.makedirs(output_dir, exist_ok=True)
                output_file = os.path.join(output_dir, file)

                try:
                    # Load the .npy file
                    image = np.load(input_file)
                    # Convert to PIL image and resize
                    resized_image = Image.fromarray(image).resize(target_size, Image.Resampling.LANCZOS)
                    # Save the resized image back as .npy
                    np.save(output_file, np.array(resized_image))
                except Exception as e:
                    print(f"Error resizing {input_file}: {e}")

# Resize RP images for each trial
for input_path, output_path in zip(rp_image_paths, output_paths):
    resize_rp_images(input_path, output_path)

print("Resizing of RP images is complete.")

Resizing of RP images is complete.


# Video frames processing

In [11]:
import os
import cv2
import numpy as np
from zipfile import ZipFile

# Paths
trial_paths = {
    "trial1": "D:\\UP-FALL",
    "trial2": "D:\\UP-FALL\\trial2",
    "trial3": "D:\\UP-FALL\\trial3"
}

output_base_path = "D:\\UP-FALL\\v2"
os.makedirs(output_base_path, exist_ok=True)

# Target size for resizing
target_size = (32, 32)

# Function to extract and process frames for summed differences
def process_video_summed_diff(trial, subject, activity, trial_path, output_path, window_size, overlap):
    zip_file = os.path.join(trial_path, f"Subject{subject}Activity{activity}Trial{trial}Camera1.zip")
    
    # Check if the ZIP file exists
    if not os.path.exists(zip_file):
        print(f"Missing video data for Subject {subject}, Activity {activity}, Trial {trial}")
        return
    
    # Temporary extraction path
    temp_extract_path = os.path.join(trial_path, "temp_frames")
    os.makedirs(temp_extract_path, exist_ok=True)

    try:
        # Extract frames
        with ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(temp_extract_path)

        # Get all image files in sorted order
        frame_files = sorted([os.path.join(temp_extract_path, f) for f in os.listdir(temp_extract_path) if f.endswith('.jpg') or f.endswith('.png')])

        if len(frame_files) < 2:
            print(f"Not enough frames for Subject {subject}, Activity {activity}, Trial {trial}")
            return

        # Determine window length and overlap in terms of frames
        sampling_rate = 20  # Assuming 20 frames per second
        window_frames = int(window_size * sampling_rate)
        overlap_frames = int(overlap * window_frames)

        # Create output directory
        activity_output_path = os.path.join(output_path, f"Subject{subject}_Activity{activity}_Trial{trial}")
        os.makedirs(activity_output_path, exist_ok=True)

        # Process frames in windows
        start = 0
        window_idx = 0
        while start + window_frames <= len(frame_files):
            summed_diff = None

            # Compute summed differences for frames in the window
            for i in range(start, start + window_frames - 1):
                frame1 = cv2.imread(frame_files[i], cv2.IMREAD_GRAYSCALE)
                frame2 = cv2.imread(frame_files[i + 1], cv2.IMREAD_GRAYSCALE)
                if frame1 is not None and frame2 is not None:
                    diff = cv2.absdiff(frame2, frame1)
                    if summed_diff is None:
                        summed_diff = np.zeros_like(diff, dtype=np.float32)
                    summed_diff += diff.astype(np.float32)

            if summed_diff is not None:
                # Normalize and resize the summed difference image
                summed_diff = cv2.normalize(summed_diff, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
                resized_summed_diff = cv2.resize(summed_diff, target_size, interpolation=cv2.INTER_AREA)

                # Save the resulting image
                output_file = os.path.join(activity_output_path, f"window{window_idx}_summed_diff.jpg")
                cv2.imwrite(output_file, resized_summed_diff)

            # Move the window forward
            start += overlap_frames
            window_idx += 1

    except Exception as e:
        print(f"Error processing video for Subject {subject}, Activity {activity}, Trial {trial}: {e}")
    finally:
        # Clean up temporary files
        for f in os.listdir(temp_extract_path):
            os.remove(os.path.join(temp_extract_path, f))
        os.rmdir(temp_extract_path)

# Process all trials
for trial_name, trial_path in trial_paths.items():
    output_path = os.path.join(output_base_path, f"{trial_name}_summed_diff_images")
    os.makedirs(output_path, exist_ok=True)

    for subject in range(1, 18):  # Subjects 1 to 17
        for activity in range(1, 12):  # Activities 1 to 11
            # Set windowing parameters based on activity duration
            if activity in [1, 2, 3, 4, 5, 9]:  # Short activities (10s)
                window_size = 3  # seconds
                overlap = 0.5
            else:  # Long activities (30-60s)
                window_size = 5  # seconds
                overlap = 0.5

            process_video_summed_diff(trial_name[-1], subject, activity, trial_path, output_path, window_size, overlap)

print("Summed difference images for all trials have been created and saved.")

Missing video data for Subject 8, Activity 11, Trial 2
Missing video data for Subject 8, Activity 11, Trial 3
Summed difference images for all trials have been created and saved.


# Generating GAF images

In [13]:
import os
import numpy as np
import pandas as pd
from pyts.image import GramianAngularField
from sklearn.preprocessing import MinMaxScaler

# Paths
dataset_paths = {
    "trial1": "D:\\UP-FALL",
    "trial2": "D:\\UP-FALL\\trial2",
    "trial3": "D:\\UP-FALL\\trial3"
}
output_base_path = "D:\\UP-FALL\\v2"
os.makedirs(output_base_path, exist_ok=True)

# Columns
columns = [
    "Timestamp",
    "AnkleAccelerometer_x", "AnkleAccelerometer_y", "AnkleAccelerometer_z",
    "AnkleAngularVelocity_x", "AnkleAngularVelocity_y", "AnkleAngularVelocity_z",
    # Other columns skipped for simplicity
]

# Generate GAF
def generate_gaf(data, output_path, image_size=32):
    try:
        gaf = GramianAngularField(image_size=image_size, method='summation')
        gaf_image = gaf.fit_transform(data.reshape(1, -1))[0]
        np.save(output_path, gaf_image)
    except Exception as e:
        print(f"Error generating GAF for {output_path}: {e}")

# Process Sensor Data
def process_sensor_data(trial, subject, activity, trial_path, output_path, window_size, overlap):
    csv_file = os.path.join(trial_path, f"Subject{subject}Activity{activity}Trial{trial}.csv")
    
    # Check if the file exists
    if not os.path.exists(csv_file):
        print(f"File not found: {csv_file}")
        return

    # Load sensor data
    try:
        data = pd.read_csv(csv_file, skiprows=2, names=columns, usecols=range(7))  # Adjust column indices as needed
    except Exception as e:
        print(f"Error processing {csv_file}: {e}")
        return

    # Compute magnitudes for accelerometer and angular velocity
    acc_magnitude = np.sqrt(data["AnkleAccelerometer_x"]**2 +
                            data["AnkleAccelerometer_y"]**2 +
                            data["AnkleAccelerometer_z"]**2)
    gyro_magnitude = np.sqrt(data["AnkleAngularVelocity_x"]**2 +
                             data["AnkleAngularVelocity_y"]**2 +
                             data["AnkleAngularVelocity_z"]**2)

    # **Handle Empty Data**
    if acc_magnitude.empty or gyro_magnitude.empty:
        print(f"No valid sensor data for Subject {subject}, Activity {activity}, Trial {trial}")
        return

    # Normalize magnitudes
    try:
        scaler = MinMaxScaler((0, 1))
        acc_magnitude = scaler.fit_transform(acc_magnitude.values.reshape(-1, 1)).flatten()
        gyro_magnitude = scaler.fit_transform(gyro_magnitude.values.reshape(-1, 1)).flatten()
    except ValueError as e:
        print(f"Error normalizing data for Subject {subject}, Activity {activity}, Trial {trial}: {e}")
        return

    # Determine window length in rows
    sampling_rate = 20
    window_rows = int(window_size * sampling_rate)
    overlap_rows = int(overlap * window_rows)

    # Create output directory
    activity_output_path = os.path.join(output_path, f"Subject{subject}_Activity{activity}_Trial{trial}")
    os.makedirs(activity_output_path, exist_ok=True)

    # Generate GAF for each window
    start = 0
    window_idx = 0
    while start + window_rows <= len(acc_magnitude):
        acc_window = acc_magnitude[start:start + window_rows]
        gyro_window = gyro_magnitude[start:start + window_rows]

        # Check for empty windows
        if len(acc_window) == 0 or len(gyro_window) == 0:
            print(f"Skipping empty window for Subject {subject}, Activity {activity}, Trial {trial}")
            start += overlap_rows
            window_idx += 1
            continue

        acc_output_file = os.path.join(activity_output_path, f"window{window_idx}_acc.npy")
        gyro_output_file = os.path.join(activity_output_path, f"window{window_idx}_gyro.npy")
        generate_gaf(acc_window, acc_output_file)
        generate_gaf(gyro_window, gyro_output_file)

        # Move window forward
        start += overlap_rows
        window_idx += 1


# Generate GAF for all trials
for trial_name, trial_path in dataset_paths.items():
    output_path = os.path.join(output_base_path, f"{trial_name}_sensor_gaf_images")
    os.makedirs(output_path, exist_ok=True)

    for subject in range(1, 18):
        for activity in range(1, 12):
            if activity in [1, 2, 3, 4, 5, 9]:
                process_sensor_data(trial_name[-1], subject, activity, trial_path, output_path, 3, 0.5)
            else:
                process_sensor_data(trial_name[-1], subject, activity, trial_path, output_path, 5, 0.5)

print("GAF images have been created.")

No valid sensor data for Subject 8, Activity 11, Trial 2
No valid sensor data for Subject 8, Activity 11, Trial 3
GAF images have been created.


In [14]:
import cv2
import numpy as np
import os

# Paths
gaf_paths = {
    "trial1": "D:\\UP-FALL\\v2\\trial1_sensor_gaf_images",
    "trial2": "D:\\UP-FALL\\v2\\trial2_sensor_gaf_images",
    "trial3": "D:\\UP-FALL\\v2\\trial3_sensor_gaf_images"
}
summed_diff_paths = {
    "trial1": "D:\\UP-FALL\\v2\\trial1_summed_diff_images",
    "trial2": "D:\\UP-FALL\\v2\\trial2_summed_diff_images",
    "trial3": "D:\\UP-FALL\\v2\\trial3_summed_diff_images"
}
fused_output_path = "D:\\UP-FALL\\v2\\fused_gaf_sd"
os.makedirs(fused_output_path, exist_ok=True)

# Fuse GAF with summed difference images
def fuse_images(gaf_path, sd_path, trial_output_path):
    for root, _, files in os.walk(sd_path):
        for file in files:
            if file.endswith('.jpg'):
                sd_image_path = os.path.join(root, file)
                gaf_folder = os.path.relpath(root, sd_path)
                gaf_folder_path = os.path.join(gaf_path, gaf_folder)

                # Check if corresponding GAF files exist
                gaf_acc_path = os.path.join(gaf_folder_path, file.replace("summed_diff.jpg", "acc.npy"))
                gaf_gyro_path = os.path.join(gaf_folder_path, file.replace("summed_diff.jpg", "gyro.npy"))

                if not os.path.exists(gaf_acc_path) or not os.path.exists(gaf_gyro_path):
                    print(f"Missing GAF files for {file}")
                    continue

                # Load GAF and SD images
                sd_image = cv2.imread(sd_image_path, cv2.IMREAD_GRAYSCALE)
                gaf_acc = np.load(gaf_acc_path)
                gaf_gyro = np.load(gaf_gyro_path)

                # Stack as channels
                fused_image = np.stack([sd_image, gaf_acc, gaf_gyro], axis=-1)

                # Save fused image
                output_dir = os.path.join(trial_output_path, gaf_folder)
                os.makedirs(output_dir, exist_ok=True)
                output_file = os.path.join(output_dir, file.replace("summed_diff.jpg", "fused.npy"))
                np.save(output_file, fused_image)

# Fuse for each trial
for trial_name in ["trial1", "trial2", "trial3"]:
    fuse_images(gaf_paths[trial_name], summed_diff_paths[trial_name], os.path.join(fused_output_path, f"{trial_name}_fused_images"))

print("Fusion of GAF and Summed Difference Images is complete.")

Fusion of GAF and Summed Difference Images is complete.


# Adding additional sensor data (wrist acceleration and angular velocity magnitudes converted into RP images)

In [18]:
# Function to process wrist sensor data only
def process_wrist_sensor_data(trial, subject, activity, trial_path, output_path, window_size, overlap):
    csv_file = os.path.join(trial_path, f"Subject{subject}Activity{activity}Trial{trial}.csv")
    if not os.path.exists(csv_file):
        print(f"File not found: {csv_file}")
        return

    try:
        # Load CSV with explicitly defined columns and skip unnecessary rows
        data = pd.read_csv(csv_file, skiprows=2, names=columns, usecols=range(46))
    except Exception as e:
        print(f"Error processing {csv_file}: {e}")
        return

    # Compute magnitudes for wrist sensors
    acc_magnitude = np.sqrt(data["WristAccelerometer_x"]**2 + 
                            data["WristAccelerometer_y"]**2 + 
                            data["WristAccelerometer_z"]**2)
    gyro_magnitude = np.sqrt(data["WristAngularVelocity_x"]**2 + 
                             data["WristAngularVelocity_y"]**2 + 
                             data["WristAngularVelocity_z"]**2)

    # **Handle Empty Data**
    if acc_magnitude.empty or gyro_magnitude.empty:
        print(f"No valid sensor data for Subject {subject}, Activity {activity}, Trial {trial}")
        return

    # Normalize magnitudes
    scaler = MinMaxScaler(feature_range=(0, 1))
    try:
        acc_magnitude = scaler.fit_transform(acc_magnitude.values.reshape(-1, 1)).flatten()
        gyro_magnitude = scaler.fit_transform(gyro_magnitude.values.reshape(-1, 1)).flatten()
    except ValueError as e:
        print(f"Skipping due to empty or invalid data for Subject {subject}, Activity {activity}, Trial {trial}: {e}")
        return

    # Determine window size and overlap
    sampling_rate = 20
    window_rows = int(window_size * sampling_rate)
    overlap_rows = int(overlap * window_rows)

    # Create output directory
    activity_output_path = os.path.join(output_path, f"Subject{subject}_Activity{activity}_Trial{trial}")
    os.makedirs(activity_output_path, exist_ok=True)

    # Generate RP images for each window
    start = 0
    window_idx = 0
    while start + window_rows <= len(acc_magnitude):
        acc_window = acc_magnitude[start:start + window_rows]
        gyro_window = gyro_magnitude[start:start + window_rows]

        if len(acc_window) == 0 or len(gyro_window) == 0:
            start += overlap_rows
            window_idx += 1
            continue

        acc_output_file = os.path.join(activity_output_path, f"window{window_idx}_wrist_acc.npy")
        gyro_output_file = os.path.join(activity_output_path, f"window{window_idx}_wrist_gyro.npy")
        try:
            generate_recurrence_plot(acc_window, acc_output_file)
            generate_recurrence_plot(gyro_window, gyro_output_file)
        except Exception as e:
            print(f"Error generating RP for wrist data in {activity_output_path}: {e}")

        start += overlap_rows
        window_idx += 1

# Process all trials
for trial_name, trial_path in dataset_paths.items():
    output_path = os.path.join(output_base_path, f"{trial_name}_sensor_rp_images_wrist")
    os.makedirs(output_path, exist_ok=True)

    for subject in range(1, 18):
        for activity in range(1, 12):
            if activity in [1, 2, 3, 4, 5, 9]:
                window_size = 3
                overlap = 0.5
            else:
                window_size = 5
                overlap = 0.5

            process_wrist_sensor_data(trial_name[-1], subject, activity, trial_path, output_path, window_size, overlap)

print("RP images for wrist sensors created.")

No valid sensor data for Subject 8, Activity 11, Trial 2
No valid sensor data for Subject 8, Activity 11, Trial 3
RP images for wrist sensors created.


In [19]:
import os
import numpy as np
from PIL import Image

# Paths for RP wrist data
wrist_rp_paths = [
    "D:\\UP-FALL\\v2\\trial1_sensor_rp_images_wrist",
    "D:\\UP-FALL\\v2\\trial2_sensor_rp_images_wrist",
    "D:\\UP-FALL\\v2\\trial3_sensor_rp_images_wrist"
]

resized_output_paths = [
    "D:\\UP-FALL\\v2\\trial1_sensor_rp_images_wrist_resized",
    "D:\\UP-FALL\\v2\\trial2_sensor_rp_images_wrist_resized",
    "D:\\UP-FALL\\v2\\trial3_sensor_rp_images_wrist_resized"
]

target_size = (32, 32)

# Ensure output directories exist
for path in resized_output_paths:
    os.makedirs(path, exist_ok=True)

# Resize function
def resize_rp_images(input_path, output_path):
    for root, _, files in os.walk(input_path):
        for file in files:
            if file.endswith('.npy'):
                input_file = os.path.join(root, file)
                relative_path = os.path.relpath(root, input_path)
                output_dir = os.path.join(output_path, relative_path)
                os.makedirs(output_dir, exist_ok=True)
                output_file = os.path.join(output_dir, file)

                try:
                    image = np.load(input_file)
                    resized_image = Image.fromarray(image).resize(target_size, Image.Resampling.LANCZOS)
                    np.save(output_file, np.array(resized_image))
                except Exception as e:
                    print(f"Error resizing {input_file}: {e}")

# Resize wrist RP images
for input_path, output_path in zip(wrist_rp_paths, resized_output_paths):
    resize_rp_images(input_path, output_path)

print("Resizing of RP images for wrist is complete.")

Resizing of RP images for wrist is complete.


In [20]:
import os
import numpy as np

# Paths
fused_ankle_sd_paths = {
    "trial1": "D:\\UP-FALL\\v2\\fused_images_rp_sd_ankle\\trial1_fused_images",
    "trial2": "D:\\UP-FALL\\v2\\fused_images_rp_sd_ankle\\trial2_fused_images",
    "trial3": "D:\\UP-FALL\\v2\\fused_images_rp_sd_ankle\\trial3_fused_images"
}
wrist_rp_paths = {
    "trial1": "D:\\UP-FALL\\v2\\trial1_sensor_rp_images_wrist_resized",
    "trial2": "D:\\UP-FALL\\v2\\trial2_sensor_rp_images_wrist_resized",
    "trial3": "D:\\UP-FALL\\v2\\trial3_sensor_rp_images_wrist_resized"
}
output_base_path = "D:\\UP-FALL\\v2\\fused_images_rp_sd_ankle_wrist"
os.makedirs(output_base_path, exist_ok=True)

# Function to combine wrist RP with the existing fused dataset
def combine_wrist_with_fused(fused_dir, wrist_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    skipped = 0

    for root, _, files in os.walk(fused_dir):
        for file in files:
            if file.endswith('.npy'):
                fused_file_path = os.path.join(root, file)

                # Extract relative path for wrist data
                relative_path = os.path.relpath(root, fused_dir)
                wrist_dir_path = os.path.join(wrist_dir, relative_path)

                if not os.path.exists(wrist_dir_path):
                    print(f"Wrist directory not found: {wrist_dir_path}")
                    skipped += 1
                    continue

                wrist_acc_file = os.path.join(wrist_dir_path, file.replace('fused.npy', 'wrist_acc.npy'))
                wrist_gyro_file = os.path.join(wrist_dir_path, file.replace('fused.npy', 'wrist_gyro.npy'))

                if not os.path.exists(wrist_acc_file) or not os.path.exists(wrist_gyro_file):
                    print(f"Missing wrist files for {file}: {wrist_acc_file} or {wrist_gyro_file}")
                    skipped += 1
                    continue

                try:
                    # Load data
                    fused_data = np.load(fused_file_path)
                    wrist_acc = np.load(wrist_acc_file)
                    wrist_gyro = np.load(wrist_gyro_file)

                    # Ensure wrist data is 2D
                    if len(wrist_acc.shape) == 2:
                        wrist_acc = wrist_acc[..., np.newaxis]
                    if len(wrist_gyro.shape) == 2:
                        wrist_gyro = wrist_gyro[..., np.newaxis]

                    # Stack the fused and wrist data along the channel dimension
                    fused_combined = np.concatenate((fused_data, wrist_acc, wrist_gyro), axis=-1)

                    # Save the combined fused data
                    output_subject_path = os.path.join(output_dir, relative_path)
                    os.makedirs(output_subject_path, exist_ok=True)
                    output_file_path = os.path.join(output_subject_path, file)
                    np.save(output_file_path, fused_combined)
                except Exception as e:
                    print(f"Error processing {file}: {e}")
                    skipped += 1

    print(f"Skipped {skipped} files during fusion.")

# Combine data for each trial
for trial in ["trial1", "trial2", "trial3"]:
    fused_dir = fused_ankle_sd_paths[trial]
    wrist_dir = wrist_rp_paths[trial]
    output_dir = os.path.join(output_base_path, f"{trial}_fused_images")
    combine_wrist_with_fused(fused_dir, wrist_dir, output_dir)

print("Fusion completed. Combined dataset saved.")

Skipped 0 files during fusion.
Skipped 0 files during fusion.
Skipped 0 files during fusion.
Fusion completed. Combined dataset saved.
